In [11]:
# Run this if we want to debug the code. Set bpoints using pdb.set_trace()
# %pdb on
# import pdb

In [5]:
# check if venv activated
import sys
print(sys.path)

['C:\\Users\\bchenghi\\Desktop\\NUS\\FYP\\repos\\java-mutation-framework\\python-scripts\\experiment\\explainable-mutation', 'C:\\Users\\bchenghi\\anaconda3\\envs\\comment-mutation-exp\\python311.zip', 'C:\\Users\\bchenghi\\anaconda3\\envs\\comment-mutation-exp\\DLLs', 'C:\\Users\\bchenghi\\anaconda3\\envs\\comment-mutation-exp\\Lib', 'C:\\Users\\bchenghi\\anaconda3\\envs\\comment-mutation-exp', '', 'C:\\Users\\bchenghi\\anaconda3\\envs\\comment-mutation-exp\\Lib\\site-packages', 'C:\\Users\\bchenghi\\anaconda3\\envs\\comment-mutation-exp\\Lib\\site-packages\\win32', 'C:\\Users\\bchenghi\\anaconda3\\envs\\comment-mutation-exp\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\bchenghi\\anaconda3\\envs\\comment-mutation-exp\\Lib\\site-packages\\Pythonwin']


In [6]:
import gensim.downloader as api
from gensim.models import KeyedVectors
from os.path import exists

MODEL_NAME = "word2vec.model"
if 'model' not in globals():
    print("model not yet initialized")
    if exists(MODEL_NAME):
        print("Model found on system. Loading existing model...")
        model = KeyedVectors.load(MODEL_NAME)
    else:
        print("Downloading model from online...")
        model = api.load("word2vec-google-news-300")  # download the model and return as object ready for use
        model.save(MODEL_NAME) # save it locally
model.most_similar("cat") # check if working

[('cats', 0.8099379539489746),
 ('dog', 0.760945737361908),
 ('kitten', 0.7464984655380249),
 ('feline', 0.7326233983039856),
 ('beagle', 0.7150582671165466),
 ('puppy', 0.7075453996658325),
 ('pup', 0.6934291124343872),
 ('pet', 0.6891531348228455),
 ('felines', 0.6755931377410889),
 ('chihuahua', 0.6709762215614319)]

In [35]:
# NLP based
import numpy as np
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_multiple_whitespaces, remove_stopwords
from gensim.matutils import jaccard

CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_multiple_whitespaces, remove_stopwords]

def average_word_embedding(words):
    word_embeddings = [model[word] for word in words if word in model]
    if len(word_embeddings) == 0:
        return None
    return sum(word_embeddings) / len(word_embeddings)

def get_cosine_sim(avg_embedding1, avg_embedding2):
    return np.dot(avg_embedding1, avg_embedding2) / (np.linalg.norm(avg_embedding1) * np.linalg.norm(avg_embedding2))

def get_jaccard(words1, words2):
    return 1 - jaccard(set(words1), set(words2))

def get_cosine(words1, words2):
    avg_embedding1 = average_word_embedding(words1)
    avg_embedding2 = average_word_embedding(words2)
    return get_cosine_sim(avg_embedding1, avg_embedding2)

def get_jac_and_cos(words1, words2):
    return [get_jaccard(words1, words2), get_cosine(words1, words2)]

def get_jac_and_cos_from_sentences(sentence0, sentence1, sentence2):
    tokens0 = preprocess_string(sentence0, CUSTOM_FILTERS)
    tokens1 = preprocess_string(sentence1, CUSTOM_FILTERS)
    tokens2 = preprocess_string(sentence2, CUSTOM_FILTERS)
    return [get_jac_and_cos(tokens0, tokens1), get_jac_and_cos(tokens0, tokens2)]

In [74]:
# Utils
SRC = ".src"
OUT = ".output"
GOLD = ".gold"
SRC_POS = 0
OUT_POS = 1
GOLD_POS = 2

def get_lines_for_file(files):
    result = []
    for file in files:
        result.append(file.readline())
    return result

def get_comments_for_lines(lines):
    result = []
    for line in lines:
        result.append(line[line.index("\t") + 1:line.index("\n")])
    return result

def get_number_for_line(line):
    return int(line[:line.index("\t")])

def convert_to_csv(number, src_comment, gold_comment, out_comment, gold_jac_cos, out_jac_cos, err=""):
    arr =  [number, src_comment, gold_comment, out_comment, err]
    arr.extend(np.array(gold_jac_cos).flatten())
    arr.extend(np.array(out_jac_cos).flatten())
    return arr

def get_file_triplet(file_name_without_ext):
    files = [None, None, None]
    f_out = open(file_name_without_ext + OUT, 'r', encoding='utf-8')
    f_src = open(file_name_without_ext + SRC, 'r', encoding='utf-8')
    f_gold = open(file_name_without_ext + GOLD, 'r', encoding='utf-8')
    files[SRC_POS] = f_src
    files[OUT_POS] = f_out
    files[GOLD_POS] = f_gold
    return files

# Count :
    # Incomplete sentence
    # did not have changes, when gold had
    # Had changes, when gold did not
    # given both had changes:
        # Bigger difference in cosine + smaller difference in jaccard
        # Larger diff in jaccard
        # Smaller diff in cosine
    # others
class Analysis:
    def __init__(self):
        self.total = 0
        self.incomplete = 0
        self.both_no_change = 0
        self.no_change = 0
        self.better = 0
        self.smaller_jac = 0
        self.higher_cos = 0
        self.others = 0
        self.err = 0
        # 12000, 5971, 1878, 3482, 218, 203, 77, 170, 0, 1
    def __str__(self):
        return ", ".join(list(map(lambda x: str(x), [self.total, self.incomplete, self.both_no_change, self.no_change, 
                                                     self.better, self.smaller_jac, self.higher_cos, 
                                                     self.others, self.err])))

def analyse_row(row, analysis):
    analysis.total += 1
    err = row[4]
    if (err != ""):
        analysis.err += 1
        return
    out_sentence = row[3]
    if (out_sentence[-1] != "."):
        analysis.incomplete += 1
        return
    gold_sentence = row[2]
    src_sentence = row[1]
    gold_had_changes = src_sentence != gold_sentence
    out_had_changes = src_sentence != out_sentence
    if gold_had_changes and not out_had_changes:
        analysis.no_change += 1
        return
    if not gold_had_changes and not out_had_changes:
        analysis.both_no_change += 1
        return
    gold_jac = row[5]
    gold_cos = row[6]
    out_jac = row[7]
    out_cos = row[8]
    if (out_jac >= gold_jac and out_cos <= gold_cos):
        analysis.better += 1
        return
    if (out_jac < gold_jac):
        analysis.smaller_jac += 1
        return
    if out_cos > gold_cos:
        analysis.higher_cos += 1
        return
    analysis.others += 1

In [75]:
# Tests
import unittest
from numpy.testing import assert_array_equal

TEST_FILES_PATH = "./test-files"

class TestFunctions(unittest.TestCase):
    files_opened = []
    
    def setUp(self):
        self.files_opened = []

    def tearDown(self):
        for file in self.files_opened:
            file.close()

# ======================== Integration tests (could be shifted else where) =====================
    def test_get_lines_for_file(self):
        file_name = "test_e0"
        files = get_file_triplet(TEST_FILES_PATH + "/" + file_name)
        self.files_opened.extend(files)
        lines = get_lines_for_file(files)
        expected = ['989\tThis method sends and handles a connection factory, object and response handler to create a ResponseStream. If the active flag is false, an IOException is thrown. If the ErrorHandler throws a retryable error, the operation is retried, else an IOException is thrown. Finally, the response is handled and the stream is ended.\n', '989\tThis method sends and handles a connection factory, object and response handler to create a ResponseStream. If the active flag is false, an IOException is thrown. If the ErrorHandler throws a retryable error, the operation is retried, else an IOException is thrown. Finally, the response is handled and the stream is ended\n', '989\tThis method sends and handles a connection factory, object and response handler to create a ResponseStream. If the active flag is false, an HazelcastInstanceNotActiveException is thrown. If the ErrorHandler throws a retryable error, the operation is retried, else an IOException is thrown. Finally, the response is handled and the stream is ended.\n']
        self.assertEquals(lines, expected)

    def test_get_jac_and_cos_from_sentences(self): 
        sentence1 = "This is the first sentence."
        sentence2 = "This is the second sentence."
        sentence3 = "Hello world."
        jac_cos_res = get_jac_and_cos_from_sentences(sentence1, sentence2, sentence3)
        expected_jac_cos = [[0.5, 0.90152967], [0.0, 0.03786046]]
        np.testing.assert_almost_equal(jac_cos_res, expected_jac_cos, 5)

# ======================== Unit tests =====================
    def test_get_comments_for_lines(self):
        numberStr = "20"
        lines = [numberStr + "\tcomment0\n", numberStr + "\tcomment1\n", numberStr + "\tcomment2\n"]
        comments = get_comments_for_lines(lines)
        self.assertEquals(comments, ['comment0', 'comment1', 'comment2'])
    
    def test_get_number_for_line(self):
        number = 10
        numberStr = str(number)
        line = numberStr + "\tcomment0\n"
        actual_num = get_number_for_line(line)
        self.assertEquals(actual_num, number)
    
    def test_average_word_embedding(self):
        sentence1 = "This is the first sentence.".split()
        sentence2 = "This is the second sentence.".split()
        sentence3 = "Hello world.".split()
        avg_embedding1 = average_word_embedding(sentence1)
        avg_embedding2 = average_word_embedding(sentence2)
        avg_embedding3 = average_word_embedding(sentence3)
        if avg_embedding1 is not None and avg_embedding2 is not None and avg_embedding3 is not None:
            cosine_sim = get_cosine_sim(avg_embedding1, avg_embedding2)
            cosine_sim1 = get_cosine_sim(avg_embedding1, avg_embedding3)
            self.assertTrue(cosine_sim > cosine_sim1)
        else:
            self.fail("One or both of the sentences has no word embeddings.")

    def test_convert_to_csv(self):
        gold_jac_cos = [1, 0.5]
        out_jac_cos = [0, 2]
        row = convert_to_csv(10, "src_comment", "gold_comment", "out_comment", gold_jac_cos, out_jac_cos)
        self.assertEquals(row, [10,"src_comment","gold_comment","out_comment","",1.0,0.5,0,2])
    
    def test_get_jaccard(self):
        words1= ["word", "word1"]
        words2 = ["word", "word1"]
        actual = get_jaccard(words1, words2)
        self.assertEquals(actual, 1)
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_average_word_embedding (__main__.TestFunctions.test_average_word_embedding) ... ok
test_convert_to_csv (__main__.TestFunctions.test_convert_to_csv) ... C:\Users\bchenghi\AppData\Local\Temp\ipykernel_19856\2120339883.py:66: DeprecationWarning: Please use assertEqual instead.
  self.assertEquals(row, [10,"src_comment","gold_comment","out_comment","",1.0,0.5,0,2])
ok
test_get_comments_for_lines (__main__.TestFunctions.test_get_comments_for_lines) ... ok
test_get_jac_and_cos_from_sentences (__main__.TestFunctions.test_get_jac_and_cos_from_sentences) ... ok
test_get_jaccard (__main__.TestFunctions.test_get_jaccard) ... ok
test_get_lines_for_file (__main__.TestFunctions.test_get_lines_for_file) ... ok
test_get_number_for_line (__main__.TestFunctions.test_get_number_for_line) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.061s

OK


In [76]:
import csv
import os
import json

PATH = "C:\\Users\\bchenghi\\Desktop\\NUS\\FYP\\explainable-data\\experiment\\comment_mutate\\prediction"
PATH_TO_TEST_JSONL = "C:\\Users\\bchenghi\\Desktop\\NUS\\FYP\\explainable-data\\data\\comment_mutate\\test.jsonl"
CUSTOM_DELIM = ";"
file_names = ["test_best-bleu"]
ls_of_tokens = []

# Open the JSONL file for reading
with open(PATH_TO_TEST_JSONL, 'r', encoding='utf-8') as jsonl_file:

    # Read each line (which contains a single JSON object)
    for line in jsonl_file:

        # Convert the JSON string to a Python object
        json_object = json.loads(line)
        tokens = json_object["code_tokens"]
        ls_of_tokens.append(tokens)
        
def get_line_count(file_path):
    f = open(file_path + SRC, "r", encoding='utf-8')
    line_count = sum(1 for x in f)
    f.close()
    return line_count

for i in range(0, 10):
    file_name = "test_e" + str(i)
    file_names.append(file_name)

final_analysis = Analysis()

for i, file_name in enumerate(file_names):
    csv_file_name = "jac_cos_result_" + str(i) + ".csv"
    if (os.path.exists(csv_file_name)):
        os.remove(csv_file_name)
    with open(csv_file_name, "a", newline='') as csv_file:
        csv_file.write("sep=" + CUSTOM_DELIM + os.linesep)
        writer = csv.writer(csv_file, delimiter=CUSTOM_DELIM)
        final_path = PATH + "\\" + file_name
        files = get_file_triplet(final_path)
        line_cnt = 0
        for _ in range(get_line_count(final_path)):
            try:
                lines = get_lines_for_file(files)
                number = get_number_for_line(lines[0])
                line_cnt += 1
                comments = get_comments_for_lines(lines)
                at_least_one = False
                for tokens in ls_of_tokens:
                    found = True
                    for token in tokens:
                        if token not in comments[0]:
                            found = False
                            break
                    if found:
                        at_least_one = True
                        break
                if not at_least_one:
                    continue
                gold_jac_and_cos = get_jac_and_cos(comments[SRC_POS], comments[GOLD_POS])
                out_jac_and_cos = get_jac_and_cos(comments[SRC_POS], comments[OUT_POS])
                csv_row = convert_to_csv(number, comments[SRC_POS], comments[GOLD_POS], comments[OUT_POS], gold_jac_and_cos, out_jac_and_cos)
                analyse_row(csv_row, final_analysis)
                writer.writerow(csv_row)
            except Exception as e:
                csv_row = convert_to_csv(number, "", "", "", [0,0], [0,0], str(e))
                analyse_row(csv_row, final_analysis)
                writer.writerow(csv_row)
        for file in files:
            file.close()
print(final_analysis)

2000, 973, 359, 560, 63, 30, 14, 0, 1
